Out of date. first round not trained with adam + weight decay. models dont even exist anymore. Delete eventually.

In [1]:
import os
import torch

from models import BaseCNN, VGG_Model, ResNet18, ResBlock
from parsing_utils import make_parser

from cnn_driver import test, get_dataloaders, get_deep_features
from benchmarks import run_benchmarks





/home/rachel/anaconda3/envs/ornet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[hparams table - google sheets link](https://docs.google.com/spreadsheets/d/1XOCA3YayXT9NfAM0j9SgO9KCypA4LZj766u6BDgU_UE/edit?usp=sharing)

[tensorboard training data - adam](https://tensorboard.dev/experiment/BMkn3PX8RAe6QrzwpWLd4g/)


In [2]:
args, _ = make_parser()
args.input_dir = "/data/ornet/single_cells_cnns"
device = 'cpu' if args.cuda == 0 or not torch.cuda.is_available() else 'cuda'
device = torch.device(device)

if device == 'cuda':
    torch.cuda.manual_seed_all(73)
else:
    torch.manual_seed(73)

path_to_intermediates = "/data/ornet/gmm_intermediates"
accept_list = []
for subdir in args.classes:
    path = os.path.join(path_to_intermediates, subdir)
    files = os.listdir(path)
    accept_list.extend([x.split(".")[0]
                        for x in files if 'normalized' in x])

### Resnet

Best Global model
Hparams:    
- Lr = 0.0001
- batch_size = 64
- Shuffle =	0	


In [3]:
model = ResNet18(in_channels=2, resblock=ResBlock, outputs=3)
model.to(device)
args.roi = False
train_dataloader, test_dataloader, val_dataloader = get_dataloaders(
    args, accept_list, resize=212)

args.save_model = "/home/rachel/ornet/models/glob_res_adam_0.0001_64_False_500.pth"
checkpoint = torch.load(args.save_model)
model.load_state_dict(checkpoint['state_dict'])
print("Testing")
test(args, model, test_dataloader, device=device)

loader_dict = {
            "train": train_dataloader, "test": test_dataloader, "val": val_dataloader
        }
feats = get_deep_features(args, model, loader_dict, device=device)
run_benchmarks(feats)

Using global image inputs
/data/ornet/single_cells_cnns
Testing
Accuracy: tensor(0.5833, device='cuda:0')
['control', 'mdivi', 'llo']
[[0 2 1]
 [0 3 1]
 [1 0 4]]
Getting deep features
best performance at epoch 29
RandomForestClassifier()  accuracy: 0.6666666666666666
[[0 0 1]
 [0 4 0]
 [3 0 4]]
KNeighborsClassifier() accuracy: 0.5833333333333334
[[0 0 1]
 [2 4 1]
 [1 0 3]]
DecisionTreeClassifier() accuracy: 0.5833333333333334
[[0 1 1]
 [2 3 0]
 [1 0 4]]


Best Local model
Hparams:    
- Lr = 0.001
- batch_size = 64
- Shuffle =	1


In [4]:
## retrain
model = ResNet18(in_channels=2, resblock=ResBlock, outputs=3)
model.to(device)
args.roi = True
train_dataloader, test_dataloader, val_dataloader = get_dataloaders(
    args, accept_list, resize=228)
args.save_model = "/home/rachel/ornet/models/roi_res_adam_0.001_64_True_300.pth"
checkpoint = torch.load(args.save_model)
model.load_state_dict(checkpoint['state_dict'])
print("Testing")
test(args, model, test_dataloader, device=device)
loader_dict = {
            "train": train_dataloader, "test": test_dataloader, "val": val_dataloader
        }
feats = get_deep_features(args, model, loader_dict, device=device)
run_benchmarks(feats)

Using ROI inputs
/data/ornet/single_cells_cnns
Testing
Accuracy: tensor(0.4167, device='cuda:0')
['control', 'mdivi', 'llo']
[[1 1 1]
 [1 1 2]
 [2 0 3]]
Getting deep features
best performance at epoch 22
RandomForestClassifier()  accuracy: 0.5
[[1 1 1]
 [1 1 0]
 [1 2 4]]
KNeighborsClassifier() accuracy: 0.6666666666666666
[[3 1 1]
 [0 1 0]
 [0 2 4]]
DecisionTreeClassifier() accuracy: 0.4166666666666667
[[0 2 0]
 [1 0 0]
 [2 2 5]]


## Basic CNN

Best Global model
- 0.001
- 64
- False

In [5]:
model = BaseCNN()
model.to(device)
args.roi = False
train_dataloader, test_dataloader, val_dataloader = get_dataloaders(
    args, accept_list, resize=28)

args.save_model = "/home/rachel/ornet/models/glob_base_adam_0.001_64_False_300.pth"
checkpoint = torch.load(args.save_model)
model.load_state_dict(checkpoint['state_dict'])
print("Testing")
test(args, model, test_dataloader, device=device)

loader_dict = {
            "train": train_dataloader, "test": test_dataloader, "val": val_dataloader
        }
feats = get_deep_features(args, model, loader_dict, device=device)
run_benchmarks(feats)

Using global image inputs
/data/ornet/single_cells_cnns
Testing
Accuracy: tensor(0.5000, device='cuda:0')
['control', 'mdivi', 'llo']
[[2 0 1]
 [2 2 0]
 [3 0 2]]
Getting deep features
best performance at epoch 284
RandomForestClassifier()  accuracy: 0.6666666666666666
[[2 1 1]
 [0 2 0]
 [1 1 4]]
KNeighborsClassifier() accuracy: 0.5
[[0 1 2]
 [0 3 0]
 [3 0 3]]
DecisionTreeClassifier() accuracy: 0.6666666666666666
[[1 0 1]
 [1 3 0]
 [1 1 4]]


Best on Local

- 0.0001
- 64
- False

In [6]:
model = BaseCNN()
model.to(device)
args.roi = True
train_dataloader, test_dataloader, val_dataloader = get_dataloaders(
    args, accept_list, resize=28)

args.save_model = "/home/rachel/ornet/models/roi_base_adam_0.0001_64_False_300.pth"
checkpoint = torch.load(args.save_model)
model.load_state_dict(checkpoint['state_dict'])
print("Testing")
test(args, model, test_dataloader, device=device)

loader_dict = {
            "train": train_dataloader, "test": test_dataloader, "val": val_dataloader
        }
feats = get_deep_features(args, model, loader_dict, device=device)
run_benchmarks(feats)

Using ROI inputs
/data/ornet/single_cells_cnns
Testing
Accuracy: tensor(0.3333, device='cuda:0')
['control', 'mdivi', 'llo']
[[0 1 2]
 [1 0 3]
 [1 0 4]]
Getting deep features
best performance at epoch 291
RandomForestClassifier()  accuracy: 0.4166666666666667
[[0 1 0]
 [1 0 0]
 [2 3 5]]
KNeighborsClassifier() accuracy: 0.4166666666666667
[[0 1 0]
 [1 0 0]
 [2 3 5]]
DecisionTreeClassifier() accuracy: 0.3333333333333333
[[0 2 1]
 [1 0 0]
 [2 2 4]]
